<a href="https://colab.research.google.com/github/adityasingh345/image-caption-generator/blob/main/Image_captioning_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
adityajn105_flickr8k_path = kagglehub.dataset_download('adityajn105/flickr8k')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# sequneces is used to create a custom data generator for training batches
from tensorflow.keras.utils import Sequence
# to_categorical is used for one-hot  encoding categorical labels
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
# Keras layers for building different parts of the neural network, like convolutions, dense layers, and activation functions
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
# Additional layers for embedding, LSTM (for sequence modeling), and layer combinations
from tensorflow.keras.layers import Embedding, LSTM, add, Concatenate, Reshape, concatenate, Bidirectional

# Pre-trained models (VGG16, ResNet50, DenseNet201) from Keras applications to use as feature extractors
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet201

# Adam optimizer for training models, widely used for its adaptive learning rate
from tensorflow.keras.optimizers import Adam

# Callbacks to monitor training, such as saving the best model, early stopping, and reducing learning rate on plateau
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Warnings module to filter out unwanted warnings during execution
import warnings

# Matplotlib for creating visualizations (e.g., loss curves, model performance)
import matplotlib.pyplot as plt

# Seaborn for statistical data visualization, often used to create better-looking plots
import seaborn as sns

# textwrap for wrapping long text into shorter lines, useful for displaying captions or summaries
from textwrap import wrap

# Configuring the plotting style and font sizes for better readability of graphs

plt.rcParams['font.size'] = 12  # Set the font size for matplotlib plots
sns.set_style("dark")  # Set a dark style for seaborn plots
warnings.filterwarnings('ignore')  # Ignore warnings for cleaner output


In [ ]:
image_path = '/kaggle/input/flickr8k/Images'

In [ ]:
data = pd.read_csv('/kaggle/input/flickr8k/captions.txt')
data.head()

In [ ]:
def readImg(path, img_size = 224):
    img = load_img(path, color_mode='rgb', target_size = (img_size, img_size))
    img = img_to_array(img)
    img = img / 255.

    return img

def display_images(df):
    df = df.reset_index(drop = True)
    plt.figure(figsize = (20 ,20))
    n = 0
    for i in range(15):
        n+= 1
        plt.subplot(5, 5, n)
        plt.subplots_adjust(hspace = 0.7 , wspace = 0.3)


        image = readImg(f"/kaggle/input/flickr8k/Images/{df.image[i]}")
        plt.imshow(image)
        plt.title("\n".join(wrap(df.caption[i], 20)))

        plt.axis("off")


In [ ]:
display_images(data.sample(15))

In [ ]:
# now comes the part of text pre processing
def text_preprocessing(data):
    # In Python, lambda is a way to create a small anonymous function
    #(a function without a name) in a single line. It's often used when
    #you need a simple function for a short task, like in apply().
    data['caption'] = data['caption'].apply(lambda x: x.lower())
    data['caption'] = data['caption'].apply(lambda x: x.replace("[^A-Za-z]", ""))
    data['caption'] = data['caption'].apply(lambda x: x.replace("\s+", " "))
    data['caption'] = data['caption'].apply(lambda x: x.replace("startseq", "").replace("endseq", "").strip())
    data['caption'] = data['caption'].apply(lambda x: " ".join([word for word in x.split() if len(word)>1]))
    data['caption'] = "startseq " + data['caption']+" endseq"
    return data

In [ ]:
data = text_preprocessing(data)
captions = data['caption'].tolist()
captions[:10]

In [ ]:
# now we will tokenize
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in captions)

images = data['image'].unique().tolist()
nimages = len(images)

split_index = round(0.85*nimages)
train_images = images[:split_index]
val_images = images[split_index:]

train = data[data['image'].isin(train_images)]
test = data[data['image'].isin(val_images)]

train.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop=True)

tokenizer.texts_to_sequences(captions[1])[0]

In [ ]:
# pre trained model for fetur extracting
model = DenseNet201()
# initalizing the model

fe = Model(inputs = model.input , outputs = model.layers[-2].output)

img_size = 224

features = {}

# looping through each image
for image in tqdm(data['image'].unique().tolist()):
    img = load_img(os.path.join(image_path , image), target_size = (img_size, img_size))
    img = img_to_array(img)

    img = img / 255

    img = np.expand_dims(img, axis = 0)

    feature = fe.predict(img , verbose = 0)

    features[image] = feature

In [ ]:
from tensorflow.keras.utils import Sequence, to_categorical
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

class CustomDataGenerator(Sequence):
    # """
    # Custom Data Generator to yield batches of image features and corresponding caption sequences.

    # Attributes:
    # -----------
    # df : DataFrame
    #     DataFrame containing image identifiers and their corresponding captions.
    # X_col : str
    #     Name of the column in the DataFrame that contains image identifiers.
    # y_col : str
    #     Name of the column in the DataFrame that contains captions.
    # batch_size : int
    #     Number of samples per batch.
    # directory : str
    #     Directory where the images are stored.
    # tokenizer : Tokenizer
    #     Tokenizer object used to convert text into sequences of integers.
    # vocab_size : int
    #     Size of the vocabulary for the output categorical sequences.
    # max_length : int
    #     Maximum length for the input sequences (captions) to be padded.
    # features : dict
    #     Dictionary containing image features extracted from a pre-trained model.
    # shuffle : bool, optional
    #     Whether to shuffle the data at the end of each epoch (default is True).
    # n : int
    #     Total number of samples in the DataFrame.
    # """

    def __init__(self, df, X_col, y_col, batch_size, directory, tokenizer,
                 vocab_size, max_length, features, shuffle=True):
        """Initializes the generator with the given parameters."""
        self.df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original one
        self.X_col = X_col  # Column name for image identifiers
        self.y_col = y_col  # Column name for captions
        self.directory = directory  # Directory where images are stored
        self.batch_size = batch_size  # Number of samples in each batch
        self.tokenizer = tokenizer  # Tokenizer to convert text to sequences
        self.vocab_size = vocab_size  # Size of the vocabulary for the captions
        self.max_length = max_length  # Maximum length of input sequences for padding
        self.features = features  # Pre-extracted features of the images
        self.shuffle = shuffle  # Whether to shuffle the data at the end of each epoch
        self.n = len(self.df)  # Total number of samples

    def on_epoch_end(self):
        """Shuffles the DataFrame at the end of each epoch if shuffle is True."""
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)  # Shuffle the DataFrame

    def __len__(self):
        """Returns the number of batches per epoch."""
        return self.n // self.batch_size  # Floor division to get the number of complete batches

    def __getitem__(self, index):
        """Generates one batch of data."""
        batch = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size, :]  # Get batch samples
        X1, X2, y = self.__get_data(batch)  # Get data for the batch
        return (X1, X2), y  # Return inputs and output as a tuple

    def __get_data(self, batch):
        """Generates data for a given batch of samples."""
        X1, X2, y = list(), list(), list()  # Initialize empty lists to store features, input sequences, and outputs

        images = batch[self.X_col].tolist()  # Get list of image identifiers in the batch

        for image in images:
            feature = self.features[image][0]  # Extract pre-computed image feature from the features dictionary
            captions = batch.loc[batch[self.X_col] == image, self.y_col].tolist()  # Get captions for the image
            for caption in captions:
                seq = self.tokenizer.texts_to_sequences([caption])[0]  # Convert caption to sequence of integers

                for i in range(1, len(seq)):
                    # Split the caption into input and output sequences
                    in_seq, out_seq = seq[:i], seq[i]  # Input sequence and the next word as the output
                    in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]  # Pad input sequence to max_length
                    out_seq = to_categorical([out_seq], num_classes=self.vocab_size)[0]  # Convert output to one-hot encoding

                    X1.append(feature)  # Append image feature as input
                    X2.append(in_seq)  # Append the input sequence
                    y.append(out_seq)  # Append the output (next word in sequence)

        # Convert lists to NumPy arrays for better performance
        X1, X2, y = np.array(X1), np.array(X2), np.array(y)

        return X1, X2, y  # Return image features, input sequences, and output sequences

In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Embedding, LSTM, Dropout, add, concatenate
from tensorflow.keras.models import Model

# Define two input layers
input1 = Input(shape=(1920,))  # Input for image features
# input1 = [0.23, 0.54, ..., 0.12]
input2 = Input(shape=(max_length,))  # Input for the caption sequence
# input2 = np.array([[1 , 5 , 7, 0, ]]) shape( 1 , 5) example 'start dog runs'

# Image feature processing
img_features = Dense(256, activation='relu')(input1)  # Fully connected layer to reduce dimensionality
img_features_reshaped = Reshape((1, 256), input_shape=(256,))(img_features)  # Reshape to (1, 256) to concatenate with LSTM output
# reshape converts (1, 256) into (1,1,256) so it can be concatenated with captions


# Caption (text) processing
sentence_features = Embedding(vocab_size, 256, mask_zero=False)(input2)
 # shape(1 , 5 , 256) 1 -> batch size 1-> max_length 256 -> embedding_dim # Embedding layer for input captions
merged = concatenate([img_features_reshaped, sentence_features], axis=1)  # Concatenate image features with caption sequence
sentence_features = LSTM(256)(merged)  # LSTM processes the combined features

# Combine LSTM output with image features
x = Dropout(0.5)(sentence_features)  # Dropout to prevent overfitting
x = add([x, img_features])  # Skip connection to add image features to LSTM output
x = Dense(128, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
output = Dense(vocab_size, activation='softmax')(x)  # Output layer with softmax activation to predict the next word

# Create and compile the model
caption_model = Model(inputs=[input1, input2], outputs=output)  # Define the model with two inputs and one output
caption_model.compile(loss='categorical_crossentropy', optimizer='adam')  # Compile the model with categorical crossentropy loss


from tensorflow.keras.utils import plot_model
plot_model(caption_model)

In [ ]:
caption_model.summary()

In [ ]:
# Create a training data generator
train_generator = CustomDataGenerator(
    df=train,  # DataFrame containing training data
    X_col='image',  # Column name with image identifiers
    y_col='caption',  # Column name with captions
    batch_size=64,  # Number of samples per batch
    directory=image_path,  # Path to the directory containing images
    tokenizer=tokenizer,  # Tokenizer to convert captions to sequences
    vocab_size=vocab_size,  # Total vocabulary size for one-hot encoding
    max_length=max_length,  # Maximum length for input sequences
    features=features  # Pre-computed image features
)

# Create a validation data generator
validation_generator = CustomDataGenerator(
    df=test,  # DataFrame containing validation data
    X_col='image',  # Column name with image identifiers
    y_col='caption',  # Column name with captions
    batch_size=64,  # Number of samples per batch
    directory=image_path,  # Path to the directory containing images
    tokenizer=tokenizer,  # Tokenizer to convert captions to sequences
    vocab_size=vocab_size,  # Total vocabulary size for one-hot encoding
    max_length=max_length,  # Maximum length for input sequences
    features=features  # Pre-computed image features
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Callback to save the model with the lowest validation loss
model_name = "model.keras"  # Change the file extension to '.keras'
checkpoint = ModelCheckpoint(
    model_name,  # Filepath where the model will be saved
    monitor="val_loss",  # Monitor the validation loss during training
    mode="min",  # Save the model when the validation loss is minimized
    save_best_only=True,  # Save only the model with the best validation loss
    verbose=1  # Print a message when the model is saved
)

# Callback to stop training early if the validation loss does not improve
earlystopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    min_delta=0,  # Minimum change in the monitored value to be considered an improvement
    patience=5,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,  # Print a message when early stopping is triggered
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

# Callback to reduce the learning rate if validation loss does not improve
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor the validation loss
    patience=3,  # Number of epochs with no improvement after which the learning rate will be reduced
    verbose=1,  # Print a message when the learning rate is reduced
    factor=0.2,  # Factor by which the learning rate will be reduced (new_lr = lr * factor)
    min_lr=1e-8  # Lower bound on the learning rate to avoid reducing it too much
)

In [ ]:
# Train the image captioning model
history = caption_model.fit(
    train_generator,  # Training data generator
    epochs=50,  # Maximum number of epochs to train the model
    validation_data=validation_generator,  # Validation data generator
    callbacks=[checkpoint, earlystopping, learning_rate_reduction]  # List of callbacks to monitor and control training
)

In [ ]:
def idx_to_word(integer, tokenizer):
    """
    Converts an integer (word index) back to its corresponding word
    using the tokenizer's word index.

    Parameters:
    -----------
    integer : int
        The index of the word to be converted.
    tokenizer : Tokenizer
        The tokenizer that contains the mapping of words to indices.

    Returns:
    --------
    str or None
        The word corresponding to the given index, or None if the index is not found.
    """
    for word, index in tokenizer.word_index.items():  # Loop through all word-index pairs in the tokenizer
        if index == integer:  # If the index matches the given integer
            return word  # Return the corresponding word
    return None  # If no match is found, return None

In [ ]:
def predict_caption(model, image, tokenizer, max_length, features):
    """
    Generates a caption for a given image using the trained captioning model.

    Parameters:
    -----------
    model : keras.Model
        The trained image captioning model.
    image : str
        The identifier or key for the image in the features dictionary.
    tokenizer : Tokenizer
        The tokenizer object used to convert text into sequences of integers.
    max_length : int
        The maximum allowed length for the caption.
    features : dict
        A dictionary containing pre-extracted image features for all images.

    Returns:
    --------
    str
        The generated caption for the image.
    """

    feature = features[image]  # Extract the feature for the given image
    in_text = "startseq"  # Start the caption generation with the starting token

    for i in range(max_length):  # Limit the length of the caption to max_length
        # Convert the input text so far into a sequence of integers
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # Pad the sequence to ensure it has the required max_length
        sequence = pad_sequences([sequence], max_length)

        # Predict the next word's index using the image feature and input sequence
        y_pred = model.predict([feature, sequence], verbose=0)  # Model predicts probabilities for each word in the vocabulary
        y_pred = np.argmax(y_pred)  # Choose the word with the highest probability

        word = idx_to_word(y_pred, tokenizer)  # Convert the predicted index back to a word

        if word is None:  # If no matching word is found, stop generation
            break

        in_text += " " + word  # Add the predicted word to the caption

        if word == 'endseq':  # If the 'endseq' token is predicted, end the generation
            break

    return in_text  # Return the full caption as a string

In [ ]:
samples = test.sample(20)
samples.reset_index(drop=True,inplace=True)

In [ ]:
for index,record in samples.iterrows():

    img = load_img(os.path.join(image_path,record['image']),target_size=(224,224))
    img = img_to_array(img)
    img = img/255.

    caption = predict_caption(caption_model, record['image'], tokenizer, max_length, features)
    samples.loc[index,'caption'] = caption

In [ ]:
display_images(samples)

In [ ]:
# Specify the image filename you want to test
image_name = '1024138940_f1fefbdce1.jpg'  # replace with your image name

# Check if the image exists in the DataFrame
matching_records = samples[samples['image'] == image_name]

if not matching_records.empty:
    # If a matching record exists, access it
    record = matching_records.iloc[0]
    print(f"Record found: {record}")

    # Load and preprocess the image
    img = load_img(os.path.join(image_path, record['image']), target_size=(224, 224))
    img = img_to_array(img)
    img = img / 255.0  # Normalize

    # Generate the caption using the caption model
    caption = predict_caption(caption_model, record['image'], tokenizer, max_length, features)

    # Print the predicted caption
    print(f"Predicted Caption for {image_name}: {caption}")

    # If you want to visualize the image:
    import matplotlib.pyplot as plt

    # Load and display the image
    img = load_img(os.path.join(image_path, record['image']))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted Caption: {caption}")
    plt.show()

else:
    print(f"No matching record found for {image_name}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Function to display image and caption
def display_image_and_caption(image_path, image_name, caption):
    # Load the image
    img = mpimg.imread(os.path.join(image_path, image_name))

    # Plot the image and display the caption
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')  # Turn off the axis
    plt.title(caption, fontsize=14)
    plt.show()

In [ ]:
# Example usage
image_name = '1024138940_f1fefbdce1.jpg'  # Example image name
caption = predict_caption(caption_model, image_name, tokenizer, max_length, features)

# Display the image along with the generated caption
display_image_and_caption(image_path, image_name, caption)

In [ ]:
# Example usage
image_name = '106490881_5a2dd9b7bd.jpg'  # Example image name
caption = predict_caption(caption_model, image_name, tokenizer, max_length, features)

# Display the image along with the generated caption
display_image_and_caption(image_path, image_name, caption)